In [ ]:
python3.11 -m pip install -r requirements.txt

In [ ]:
import os
import boto3
import time
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
class FILE_DOWNLOAD():
    def __init__(self):
        self.files_upload =['data.json','data2.json','data3.json']
        self.download_directory = "/tmp/"
        self.filename = 'data.zip'
        self.minio_bucket = "data"
        self.minio_object = "data-raw"
        self.obj_storage_access_key = os.getenv('OBJ_STORAGE_ACCESS_KEY', 'access-key')
        self.obj_storage_secret_key = os.getenv('OBJ_STORAGE_SECRET_KEY', 'secret-key')
        self.obj_storage_endpoint = os.getenv('OBJ_STORAGE_ENDPOINT', 'http://localhost:9000')

    def crawler(self):
        # Specify the URL
        url = "https://drive.google.com/drive/folders/1EImvKyUQ_iJCfd2h-Dxjn_sqPyDV5tFC?usp=sharing"

        # Initialize Chrome WebDriver
        options = webdriver.ChromeOptions()
        options.add_experimental_option("prefs", {
                "download.default_directory": self.download_directory,
                "download.prompt_for_download": False,
                "download.directory_upgrade": True,
                "safebrowsing.enabled": True
            })

        driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)

        for tries in range (0,3):
            # Open the URL
            driver.get(url)

            # Locate the file you want to download by XPath or CSS selector
            file_locator = "/html/body/div[3]/div/div[3]/div[2]/div[1]/div/c-wiz/div/c-wiz/div[1]/c-wiz/div[2]/c-wiz/div[1]/c-wiz/c-wiz/div/c-wiz/div/div/div/div[6]/div"
            file_element = driver.find_element(By.XPATH, file_locator)

            # Move the mouse to the file location
            action_chains = ActionChains(driver)
            action_chains.move_to_element(file_element).perform()

            # Right-click on the file to trigger download
            action_chains.click(file_element).perform()

            print('Downloading ...')

            # Wait for the file to download
            time.sleep(10)  # You might need to adjust this depending on the file size and network speed

            # Call to verify function to check file is download or not. If not, retrying to download again
            verify_success = self.file_download_verify()
            if verify_success:
                break
            else:
                print('Trying again {} time(s)'.format(tries))
        # Close the browser
        driver.quit()

    def file_download_verify(self):
        success = False
        # List a directory to check the exists of the file
        res = os.listdir(self.download_directory)
        try:
           if self.filename in res:
              print("File downloaded successfully")
              success = True
           else:
               print("File not download")
        except "file is not downloaded":
           print("file not download")
        return success

In [ ]:
    def upload_file_to_minio(self,file_path, minio_bucket, minio_object_name):
        # Object initialized
        s3c = boto3.resource('s3',
                            endpoint_url=self.obj_storage_endpoint,
                            aws_access_key_id=self.obj_storage_access_key,
                            aws_secret_access_key=self.obj_storage_secret_key,
                            config=boto3.session.Config(signature_version='s3v4'),
                            verify=False
                            )
        s3c.Bucket(minio_bucket).upload_file(file_path, minio_object_name)


In [ ]:
    def main(self):
        # Run a crawler function
        self.crawler()
        # Unzip the file
        shutil.unpack_archive(f"{self.download_directory}/{self.filename}", self.download_directory)
        # Looping to upload file
        for file in self.files_upload:
            print('Uploading file {}...'.format(file))
            self.upload_file_to_minio(f"{self.download_directory}/{file}", self.minio_bucket, f"{self.minio_object}/{file}")
            print('Uploaded')

In [ ]:
if __name__ == '__main__':#
    file_download = FILE_DOWNLOAD()
    downloading = file_download.main()
    print('Finished downloading')